<a href="https://colab.research.google.com/github/pinggu95/deep_dive_AI/blob/main/51%EC%9D%BC%EC%B0%A8_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 우리말 의학용어의 필수의학용어집

# 1. 라이브러리 설치

In [25]:
!pip install langchain
!pip install faiss-cpu
!pip install unstructured
!pip install pypdf
!pip install sentence-transformers
!pip install openai
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=3667a568658af0e087109d5500bc044d6c97785fb805eef6a47385998aefa067
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


#2. 기본 라이브러리 불러오기

In [26]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import gradio as gr
import os

#  3. PDF 문서 로딩

In [27]:
pdf_path = "/content/drive/MyDrive/Colab Notebooks/딥다이브/우리말_의학용어의_필수의학용어집을_중심으로.pdf.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

# 4. 텍스트 분할

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=10,
    separators=["\n\n", "\n", " "]
)
texts = text_splitter.split_documents(pages)


#  5. 임베딩 모델 로드

In [29]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-29-3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#  6. FAISS 벡터 데이터베이스 생성

In [30]:
vector_db = FAISS.from_documents(texts, embeddings)
retriever = vector_db.as_retriever()

#  7. LLM 연결

In [ ]:

os.environ["OPENAI_API_KEY"] = "sk-..." # API 키 입력

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

#  8. Gradio 인터페이스 구성

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 우리말 필수 의학용어 챗봇")

    chatbot = gr.Chatbot(label="RAG 기반 챗봇")
    msg = gr.Textbox(label="질문을 입력하세요")
    state = gr.State([])  # 대화 이력 저장용 상태

    def respond(message, chat_history):
        response = qa_chain.run(message)
        chat_history.append((message, response))
        return "", chat_history

    msg.submit(respond, [msg, state], [msg, chatbot])
    gr.Button("대화 초기화").click(lambda: [], None, chatbot)

demo.launch()